# ESIP Marine Data Cluster 2022-08-11

* Andrew Barna (http://github.com/DocOtak/)

## GSW-Xarray

### How it Started:
https://github.com/TEOS-10/GSW-Python/issues/84

## How it's going:
https://github.com/DocOtak/gsw-xarray

Major credit to Romain Caneill https://github.com/rcaneill

## Initial Situation
* GSW-Python is pretty good
  * Wraps the GSW-C implementation
  * Numpy ufuncs internally (super fast)
* GSW-Python has some funniness
  * The numpy ufuncs are not directly exposed
  * the match_args_return decorator was written before the numpy `__array_*` type magics
  * match_args_return makes assumptions about how the python functions will be called and what dtypes they will be called with
  * When duck type support was added to GSW-Python, it could create unexpected results for attributes

In [1]:
## upstream GSW and xarray
import gsw
import xarray as xr

ds = xr.load_dataset("33RR20220430_bottle.nc")
SA = gsw.SA_from_SP(ds.ctd_salinity, ds.pressure, ds.longitude, ds.latitude)

# the returned xr.DataArray has SA in it, but the attributes are coppied from the 
# first argument and are now not correct for the actual data values
SA

<xarray.DataArray 'ctd_salinity' (N_PROF: 149, N_LEVELS: 36)>
array([[34.79407258, 34.79135976, 34.79417296, ..., 34.79346935,
        34.78945046, 34.78161362],
       [34.78467647, 34.78447231, 34.7846391 , ..., 34.74103721,
        34.74092823, 34.74124926],
       [34.64124952, 34.64306294, 34.6746939 , ...,         nan,
                nan,         nan],
       ...,
       [35.21968957, 35.22332819, 35.22074988, ...,         nan,
                nan,         nan],
       [35.18337846, 35.22190961, 35.26873752, ..., 34.87015424,
        34.87009813, 34.87072055],
       [35.21882931, 35.24081707, 35.24604888, ..., 34.86956369,
        34.87018814, 34.86991643]])
Coordinates:
    expocode   (N_PROF) object '33RR20220430' '33RR20220430' ... '33RR20220430'
    station    (N_PROF) object '1' '1' '3' '4' '5' ... '115' '116' '116' '117'
    cast       (N_PROF) int32 1 2 1 1 1 1 1 1 4 1 1 1 ... 1 2 1 1 1 2 1 1 1 2 1
    sample     (N_PROF, N_LEVELS) object '3' '29' '5' '26' ... '4' '3' '2' '1'
    time       (N_PROF) datetime64[ns] 2022-05-01T20:43:00.000000256 ... 2022...
    latitude   (N_PROF) float64 21.0 21.0 32.51 32.41 ... 30.0 30.0 30.0 30.0
    longitude  (N_PROF) float64 140.1 140.1 133.0 133.3 ... -161.8 -161.8 -161.2
    pressure   (N_PROF, N_LEVELS) float64 12.69 12.81 ... 5.406e+03 5.832e+03
Dimensions without coordinates: N_PROF, N_LEVELS
Attributes:
    whp_name:             CTDSAL
    whp_unit:             PSS-78
    standard_name:        sea_water_practical_salinity
    units:                1
    reference_scale:      PSS-78
    C_format:             %9.4f
    C_format_source:      database
    ancillary_variables:  ctd_salinity_qc

## First Goal
* wrap gsw with an xarray aware wrapper that can set the attributes to have the correct CF attrs for what is returned by that function
* initial versions would only replace the `standard_name` and `units` attributes, eventually we decided to completely replace the attrs with our own since they are free form in the input

In [2]:
import gsw_xarray as gsw_xr

In [3]:
# exact same function signature
SA = gsw_xr.SA_from_SP(ds.ctd_salinity, ds.pressure, ds.longitude, ds.latitude)
# now attrs are completly replaced, we even name the returned xr.DataArray using the GSW standard nomenclature
SA

<xarray.DataArray 'SA' (N_PROF: 149, N_LEVELS: 36)>
array([[34.79407258, 34.79135976, 34.79417296, ..., 34.79346935,
        34.78945046, 34.78161362],
       [34.78467647, 34.78447231, 34.7846391 , ..., 34.74103721,
        34.74092823, 34.74124926],
       [34.64124952, 34.64306294, 34.6746939 , ...,         nan,
                nan,         nan],
       ...,
       [35.21968957, 35.22332819, 35.22074988, ...,         nan,
                nan,         nan],
       [35.18337846, 35.22190961, 35.26873752, ..., 34.87015424,
        34.87009813, 34.87072055],
       [35.21882931, 35.24081707, 35.24604888, ..., 34.86956369,
        34.87018814, 34.86991643]])
Coordinates:
    expocode   (N_PROF) object '33RR20220430' '33RR20220430' ... '33RR20220430'
    station    (N_PROF) object '1' '1' '3' '4' '5' ... '115' '116' '116' '117'
    cast       (N_PROF) int32 1 2 1 1 1 1 1 1 4 1 1 1 ... 1 2 1 1 1 2 1 1 1 2 1
    sample     (N_PROF, N_LEVELS) object '3' '29' '5' '26' ... '4' '3' '2' '1'
    time       (N_PROF) datetime64[ns] 2022-05-01T20:43:00.000000256 ... 2022...
    latitude   (N_PROF) float64 21.0 21.0 32.51 32.41 ... 30.0 30.0 30.0 30.0
    longitude  (N_PROF) float64 140.1 140.1 133.0 133.3 ... -161.8 -161.8 -161.2
    pressure   (N_PROF, N_LEVELS) float64 12.69 12.81 ... 5.406e+03 5.832e+03
Dimensions without coordinates: N_PROF, N_LEVELS
Attributes:
    standard_name:  sea_water_absolute_salinity
    units:          g/kg

## Release Day!
* Romain did this: https://twitter.com/rcaneill/status/1506266529184333831

## What's next?
* Immediate request was for pint support (not just from @rabernat)
* Work on that is ongoing, but almost all of it has landed in the dev version
* Minor(ish) issues with upstream gsw assumptions about how functions are called (args vs kwargs)
  * `gsw.SA_from_SP(SP, p, lon, lat)` vs `gsw.SA_from_SP(SP=SP, p=p, lon=lon, lat=lat)`
  * kwargs can be in any order
  
  
* Note that at the time of this presentation (2022-08-11) the remainder of the examples use the dev (not pypi) version of gssw-xarray

In [4]:
from pint import application_registry as ureg
# cf_xarray.units add CF specific things to the unit registry (e.g. degrees_north)
import cf_xarray.units
import pint_xarray

# pint doesn't support unitless scaling factors (e.g. /MILLE in CF would be 1e-3) just removing those vars for demo
try:
    del ds["del_carbon_13_dic"]
    del ds["del_carbon_14_dic"]
except KeyError:
    pass
ds_pint = ds.pint.quantify()
ds_pint

Magnitude,[20.99958 20.99956 32.50698 32.41284 32.2463 32.0735 31.8842 31.81008 31.63518 31.09096 30.54484 29.99674 29.9987 30.00009 30.00154 30.001 30.00048 29.99882 30.0008 29.99972 29.99974 30.00019 30.00022 30.00024 30.00014 29.99932 29.99998 29.99992 30.0 29.99958 29.99999 30.00014 30.00005 29.9999 30.00026 30.00024 30.00046 30.00014 30.00006 30.00006 30.0004 30.00002 29.99956 29.99986 29.99994 29.99942 30.00064 30.00072 29.999 29.99924 30.00048 30.0005 30.00014 30.00002 30.00064 30.0007 29.99982 29.99973 30.0006 30.00056 29.9997 29.99956 29.99862 29.99862 29.99898 30.00002 30.00048 30.00046 29.99978 29.99905 30.00002 30.0 30.00004 29.99932 30.00022 30.00026 29.99936 29.99918 30.00028 30.00036 29.99968 29.99882 29.99996 29.99996 29.99986 30.0003 29.99986 29.9999 30.00036 29.99994 29.9999 29.99992 30.0002 30.00067 30.0 30.0 30.00014 29.99946 29.99956 29.9996 30.0 29.99981 30.00002 30.00006 30.00064 29.99976 29.99976 29.99976 30.0001 30.0003 30.00022 30.00022 29.99986 30.00014 29.99986 29.9999 29.9995 30.00016 30.00066 30.0006 29.99936 29.99946 29.99992 30.0 29.99988 30.00022 29.99974 30.0004 29.99998 30.00034 30.00006 30.00008 30.00042 30.00104 29.99898 29.99898 30.00032 29.99942 29.9987 29.99872 30.00022 30.00028 30.00068 30.0007 30.00006 30.00052 30.0009 30.0009 29.9994]
Units,degrees_north
Magnitude,[140.05022 140.0502 133.02894 133.29036 133.38594 133.48432 133.6212 133.69254 133.79684 134.09372 134.45468 134.86118 135.63018 136.44697 137.16913 137.93686 138.36446 138.7947 139.65996 140.52478 141.39046 142.25622 143.17726 143.17728 143.84946 144.52378 145.19594 145.19594 145.86922 146.5425 147.21558 147.21552 147.88744 148.56094 149.2354 149.2354 149.90782 150.58137 151.25384 151.25384 151.9268 152.6001 153.27332 153.27324 153.94612 154.61926 155.29352 155.29358 155.96576 156.63904 157.31154 157.3115 157.9847 158.65838 159.32982 159.32996 159.91242 160.48916 161.0654 161.0654 161.64266 162.22014 162.79642 162.79644 163.37292 163.9499 164.52654 164.52656 165.104 165.68016 166.25786 166.25784 166.83432 167.41178 167.98852 167.9885 168.56628 169.1427 169.71904 169.71906 170.2968 170.87334 171.45088 171.45086 172.02738 172.60468 173.18174 173.18178 173.34072 173.7183 173.97906 173.97906 174.5567 175.13228 175.71416 175.71414 176.28606 176.86334 177.44032 177.44018 178.0167 178.5944 179.17116 179.17122 179.74806 -179.6744 -179.09656 -179.09658 -178.52126 -177.94418 -177.36732 -177.3673 -176.79008 -176.21256 -175.63662 -175.63662 -175.05867 -174.48204 -173.90518 -173.90516 -173.32824 -172.75192 -172.17566 -172.17588 -171.59786 -171.0212 -170.44445 -170.44552 -169.86656 -169.29024 -168.71398 -168.71396 -168.13498 -167.55852 -166.98282 -166.98282 -166.40596 -165.82834 -165.25208 -165.25207 -164.6754 -164.0982 -163.52154 -163.5215 -162.94311 -162.36716 -161.79138 -161.79136 -161.21314]
Units,degrees_east
Magnitude,[[12.689 12.81 12.842 ... 13.353 13.369 13.407] [3.9 4.0 4.0 ... 1503.8 1503.9 1504.3] [7.7 22.5 38.3 ... nan nan nan] ... [2.411922222222221 2.592183333333333 2.6606888888888895 ... nan nan nan] [3.0 21.5 40.8 ... 5265.4 5559.2 5832.0] [4.1 25.7 51.7 ... 5096.3 5405.8 5831.5]]
Units,decibar
Magnitude,[nan 1504.0 168.0 960.0 1162.0 2136.0 3058.0 4230.0 4892.0 4469.0 4505.0 4702.0 4187.0 4140.0 4345.0 4011.0 3660.0 2385.0 2234.0 2487.0 3940.0 5949.0 6001.0 6001.0 5302.0 5851.0 5881.0 5881.0 6185.0 6205.0 6221.0 6221.0 6063.0 6161.0 6185.0 6185.0 6109.0 6105.0 6074.0 6074.0 5993.0 5921.0 5844.0 5844.0 5859.0 5788.0 5807.0 5807.0 5792.0 5798.0 5258.0 5258.0 5714.0 5801.0 5735.0 5735.0 5689.0 5758.0 5902.0 5902.0 5987.0 6003.0 6015.0 6015.0 5992.0 5956.0 5888.0 5888.0 5851.0 5976.0 5904.0 5904.0 6123.0 6003.0 5887.0 5887.0 5861.0 5725.0 5678.0 5678.0 5568.0 5597.0 5392.0 5392.0 5122.0 5594.0 5247.0 5247.0 4370.0 2220.0 4599.0 4599.0 5361.0 5286.0 5167.0 5167.0 5211.0 5188.0 5252.0 5252.0 5338.0 4913.0 4488.0 4488.0 5360.0 5353.0 5428.0 5428.0 5061.0 5214.0 5354.0 5354.0 5402.0 5285.0 5286.0 5286.0 5337.0 5374.0 3683.0 368

In [5]:
# upstream GSW really doesn't like pint quantities, the following will error
SA = gsw.SA_from_SP(ds_pint.ctd_salinity, ds_pint.pressure, ds_pint.longitude, ds_pint.latitude)


TypeError: operand type(s) all returned NotImplemented from __array_ufunc__(<ufunc 'sa_from_sp'>, '__call__', <Quantity([[34.6306     34.6279     34.6307     ... 34.63       34.626
  34.6182    ]
 [34.62124484 34.62104166 34.62120767 ... 34.5610399  34.56093118
  34.56124927]
 [34.47803681 34.47981194 34.51120611 ...         nan         nan
          nan]
 ...
 [35.05409059 35.05771182 35.05514554 ...         nan         nan
          nan]
 [35.01794931 35.05627307 35.10285605 ... 34.68707094 34.68718169
  34.6877945 ]
 [35.05323569 35.07509252 35.08025015 ... 34.68636776 34.68718943
  34.68701746]], 'dimensionless')>, <Quantity([[1.26890000e+01 1.28100000e+01 1.28420000e+01 ... 1.33530000e+01
  1.33690000e+01 1.34070000e+01]
 [3.90000000e+00 4.00000000e+00 4.00000000e+00 ... 1.50380000e+03
  1.50390000e+03 1.50430000e+03]
 [7.70000000e+00 2.25000000e+01 3.83000000e+01 ...            nan
             nan            nan]
 ...
 [2.41192222e+00 2.59218333e+00 2.66068889e+00 ...           

In [6]:
# pint things
SA = gsw_xr.SA_from_SP(ds_pint.ctd_salinity, ds_pint.pressure, ds_pint.longitude, ds_pint.latitude)
SA

Magnitude,[[34.79407258054115 34.79135976347489 34.79417296178287 ... 34.793469354187934 34.78945046287356 34.78161362078254] [34.78467647131322 34.78447230551686 34.7846390988917 ... 34.74103720517914 34.74092823442572 34.741249256827764] [34.641249524067476 34.64306294385535 34.67469389919257 ... nan nan nan] ... [35.21968956538447 35.22332818596349 35.220749884308475 ... nan nan nan] [35.18337845803523 35.22190961228875 35.268737524861145 ... 34.87015423963045 34.87009813398189 34.87072055353786] [35.218829309642004 35.24081707088724 35.24604887859953 ... 34.86956368891936 34.87018813886043 34.86991642840576]]
Units,gram/kilogram
Magnitude,[20.99958 20.99956 32.50698 32.41284 32.2463 32.0735 31.8842 31.81008 31.63518 31.09096 30.54484 29.99674 29.9987 30.00009 30.00154 30.001 30.00048 29.99882 30.0008 29.99972 29.99974 30.00019 30.00022 30.00024 30.00014 29.99932 29.99998 29.99992 30.0 29.99958 29.99999 30.00014 30.00005 29.9999 30.00026 30.00024 30.00046 30.00014 30.00006 30.00006 30.0004 30.00002 29.99956 29.99986 29.99994 29.99942 30.00064 30.00072 29.999 29.99924 30.00048 30.0005 30.00014 30.00002 30.00064 30.0007 29.99982 29.99973 30.0006 30.00056 29.9997 29.99956 29.99862 29.99862 29.99898 30.00002 30.00048 30.00046 29.99978 29.99905 30.00002 30.0 30.00004 29.99932 30.00022 30.00026 29.99936 29.99918 30.00028 30.00036 29.99968 29.99882 29.99996 29.99996 29.99986 30.0003 29.99986 29.9999 30.00036 29.99994 29.9999 29.99992 30.0002 30.00067 30.0 30.0 30.00014 29.99946 29.99956 29.9996 30.0 29.99981 30.00002 30.00006 30.00064 29.99976 29.99976 29.99976 30.0001 30.0003 30.00022 30.00022 29.99986 30.00014 29.99986 29.9999 29.9995 30.00016 30.00066 30.0006 29.99936 29.99946 29.99992 30.0 29.99988 30.00022 29.99974 30.0004 29.99998 30.00034 30.00006 30.00008 30.00042 30.00104 29.99898 29.99898 30.00032 29.99942 29.9987 29.99872 30.00022 30.00028 30.00068 30.0007 30.00006 30.00052 30.0009 30.0009 29.9994]
Units,degrees_north
Magnitude,[140.05022 140.0502 133.02894 133.29036 133.38594 133.48432 133.6212 133.69254 133.79684 134.09372 134.45468 134.86118 135.63018 136.44697 137.16913 137.93686 138.36446 138.7947 139.65996 140.52478 141.39046 142.25622 143.17726 143.17728 143.84946 144.52378 145.19594 145.19594 145.86922 146.5425 147.21558 147.21552 147.88744 148.56094 149.2354 149.2354 149.90782 150.58137 151.25384 151.25384 151.9268 152.6001 153.27332 153.27324 153.94612 154.61926 155.29352 155.29358 155.96576 156.63904 157.31154 157.3115 157.9847 158.65838 159.32982 159.32996 159.91242 160.48916 161.0654 161.0654 161.64266 162.22014 162.79642 162.79644 163.37292 163.9499 164.52654 164.52656 165.104 165.68016 166.25786 166.25784 166.83432 167.41178 167.98852 167.9885 168.56628 169.1427 169.71904 169.71906 170.2968 170.87334 171.45088 171.45086 172.02738 172.60468 173.18174 173.18178 173.34072 173.7183 173.97906 173.97906 174.5567 175.13228 175.71416 175.71414 176.28606 176.86334 177.44032 177.44018 178.0167 178.5944 179.17116 179.17122 179.74806 -179.6744 -179.09656 -179.09658 -178.52126 -177.94418 -177.36732 -177.3673 -176.79008 -176.21256 -175.63662 -175.63662 -175.05867 -174.48204 -173.90518 -173.90516 -173.32824 -172.75192 -172.17566 -172.17588 -171.59786 -171.0212 -170.44445 -170.44552 -169.86656 -169.29024 -168.71398 -168.71396 -168.13498 -167.55852 -166.98282 -166.98282 -166.40596 -165.82834 -165.25208 -165.25207 -164.6754 -164.0982 -163.52154 -163.5215 -162.94311 -162.36716 -161.79138 -161.79136 -161.21314]
Units,degrees_east
Magnitude,[[12.689 12.81 12.842 ... 13.353 13.369 13.407] [3.9 4.0 4.0 ... 1503.8 1503.9 1504.3] [7.7 22.5 38.3 ... nan nan nan] ... [2.411922222222221 2.592183333333333 2.6606888888888895 ... nan nan nan] [3.0 21.5 40.8 ... 5265.4 5559.2 5832.0] [4.1 25.7 51.7 ... 5096.3 5405.8 5831.5]]
Units,decibar


## Future plans
* Somehow automatically figuring out what properties can be made and letting me just ask for them
* probably as an xarray dataset accessor which would allow things like:

  ```
  ds.gsw.SA
  ```
  
  to just have the SA values calculated automatically
  
* this isn't very easy:
  * properties I want (e.g. density) have intermediate calculations for most observed oceanographic data (e.g. I need to calculate CT and SA)
  * Many oceanographic datasets contain replicate measurements (bottle and ctd salinity, multiple salinity/temperature channels etc..)
  
* Upstream GSW improvements
  * We are testing against the entire gsw API surface as it exists, discovering some areas that could be improved
  * https://github.com/TEOS-10/GSW-Python/issues/97
  * https://github.com/TEOS-10/GSW-Python/pull/98